# First attempt at a world model
## Basic Process

In [1]:
# Settings and setup
import tensorflow as tf

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# Main Settings
ROLE = "TAGGER"

# Encoder Settings
IMG_INPUT_SHAPE=(640, 640, 3)

NUM_DBS = 8
NUM_LAYERS_PER_DB = 1
NUM_FILTERS_PER_LAYER_OF_DB = 2

DOWNSCALING_LAYERS = 4
DOWNSCALING_FACTOR = 2
OUTPUT_SIZE = 32
ENC_DROPOUT = 0.25

# PREDICTOR
PRED_INTER_LAYERS = 3
PRED_INTER_LAYER_SIZE = 64
PRED_DROPOUT = 0.25

# Action Generator Settings
ACTIONS = 4
ACT_INTER_LAYERS = 5
ACT_INTER_LAYER_SIZE = 1024
ACT_DROPOUT = 0.1

# Cost Calculator Settings
CC_INTER_LAYERS = 3
CC_INTER_LAYER_SIZE = 64
CC_DROPOUT = 0.25

2023-09-28 20:59:54.397152: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-28 20:59:54.472649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 20:59:55.220731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Encoder

In [2]:
# Image segmenter base
from tensorflow.keras import layers, models

def create_dense_block(x, num_of_layers, num_filters, kernel_size=5):
    output_layers = [x]
    for l in range(num_of_layers):
        x = layers.Conv2D(num_filters, kernel_size, activation="elu", padding="same")(x)
        output_layers.append(x)
        x = layers.Concatenate()(output_layers)
    return x

def create_transistion_down(x, layer_size_change, num_filters=None):
    if num_filters:
        x = layers.Conv2D(num_filters, 1, activation="elu", padding="same")(x)
    
    return layers.MaxPool2D(layer_size_change)(x)

# Keep track for skip connections and other things
skip_connections = []
previous_filters = [NUM_LAYERS_PER_DB * NUM_FILTERS_PER_LAYER_OF_DB]

# Num of filters: 4 (layers) * 32 (idk) + 64 (previous filters)
# Should prob figure out why this is

model_in = layers.Input(shape=IMG_INPUT_SHAPE)
x = layers.Conv2D(previous_filters[-1], 9, activation="elu", padding="same")(model_in)

for i in range(NUM_DBS):
    previous_filters.append(NUM_LAYERS_PER_DB * NUM_FILTERS_PER_LAYER_OF_DB + previous_filters[-1])
    x = create_dense_block(x, NUM_LAYERS_PER_DB, previous_filters[-1])
    skip_connections.append(x)
    x = create_transistion_down(x, 2, num_filters=(previous_filters[-1]))


previous_filters.append(NUM_LAYERS_PER_DB * NUM_FILTERS_PER_LAYER_OF_DB + previous_filters[-1])
x = create_dense_block(x, NUM_LAYERS_PER_DB, previous_filters[-1]) 

dense_net = models.Model(model_in, x)

dense_net.summary()

2023-09-28 20:59:56.239253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22226 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:18:00.0, compute capability: 8.9
2023-09-28 20:59:56.239870: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22451 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 640, 640, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 640, 640, 2)          488       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 640, 640, 4)          204       ['conv2d[0][0]']              
                                                                                                  
 concatenate (Concatenate)   (None, 640, 640, 6)          0         ['conv2d[0][0]',              
                                                                     'conv2d_1[0][0]']        

In [3]:
# Encoder (Simple downscaler)

# Added encoder to flatten dense_net
encoder = models.Sequential()
encoder.add(layers.Input(shape=IMG_INPUT_SHAPE))
encoder.add(layers.RandomFlip())
encoder.add(dense_net)
encoder.add(layers.Flatten())

for i in range(DOWNSCALING_LAYERS + 1):
    encoder.add(layers.Dense(DOWNSCALING_FACTOR**(DOWNSCALING_LAYERS-i) * OUTPUT_SIZE, activation="elu", kernel_regularizer='l2'))
    encoder.add(layers.Dropout(ENC_DROPOUT))

encoder.add(layers.BatchNormalization())

encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 640, 640, 3)       0         
                                                                 
 model (Functional)          (None, 2, 2, 38)          35780     
                                                                 
 flatten (Flatten)           (None, 152)               0         
                                                                 
 dense (Dense)               (None, 512)               78336     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

# Predictor

In [4]:
curr_shape = OUTPUT_SIZE + ACTIONS

predictor = models.Sequential()
predictor.add(layers.Input(shape=curr_shape))

# Add internal layers
for _ in range(PRED_INTER_LAYERS):
    predictor.add(layers.Dense(PRED_INTER_LAYER_SIZE, activation="elu", kernel_regularizer='l2'))
    predictor.add(layers.Dropout(PRED_DROPOUT))


predictor.add(layers.Dense(OUTPUT_SIZE, activation="elu"))

predictor.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                2368      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 32)               

# Cost Calculator

In [5]:
cost_calculator = models.Sequential()
cost_calculator.add(layers.Input(shape=OUTPUT_SIZE))

# Add internal layers
for _ in range(CC_INTER_LAYERS):
    cost_calculator.add(layers.Dense(CC_INTER_LAYER_SIZE, activation="elu", kernel_regularizer='l2'))
    cost_calculator.add(layers.Dropout(CC_DROPOUT))

cost_calculator.add(layers.Dense(CC_INTER_LAYER_SIZE//2, activation="elu"))
cost_calculator.add(layers.Dense(CC_INTER_LAYER_SIZE//4, activation="elu"))
cost_calculator.add(layers.Dense(1, activation="elu"))

cost_calculator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                2112      
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 64)                4160      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 32)               

## Action Generator

In [6]:
action_gen = models.Sequential()
action_gen.add(layers.Input(shape=OUTPUT_SIZE))

# Add internal layers
for _ in range(ACT_INTER_LAYERS):
    action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE, activation="elu"))
    action_gen.add(layers.Dropout(ACT_DROPOUT))

action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE//2, activation="elu"))
action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE//4, activation="elu"))
action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE//8, activation="elu"))
action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE//16, activation="elu"))
action_gen.add(layers.Dense(ACT_INTER_LAYER_SIZE//32, activation="elu"))
action_gen.add(layers.Dense(ACTIONS, activation="sigmoid"))
    
action_gen.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 1024)              33792     
                                                                 
 dropout_11 (Dropout)        (None, 1024)              0         
                                                                 
 dense_16 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_17 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_13 (Dropout)        (None, 1024)              0         
                                                                 
 dense_18 (Dense)            (None, 1024)             

# Training

In [7]:
# Open data and convert to tensors

In [8]:
# Settings
ENCODER_AND_COST_LR = 0.00001
PRED_LR = 0.000005
ACT_GEN_LR = 0.000001

MAX_EPOCHS = 100
BATCH_SIZE = 16

In [9]:
# Create dataset
import tensorflow as tf
import pandas as pd

with tf.device("cpu:0"):
    runs = pd.read_csv("game_runs.csv")[["Run", "Winner"]].sample(frac=1).values.tolist()
    all_frames = []
    all_next_frames = []
    all_input = []
    all_scores = []
    for run, winner in runs[:30]:
        frames = tf.cast(tf.io.parse_tensor(tf.io.read_file("runs/%d/frames.proto_tensor" % run), tf.uint8), tf.float16)/255.0
        all_frames.append(frames)
    
        if ROLE == "TAGGER":
            all_input.append(tf.io.parse_tensor(tf.io.read_file("runs/%d/tagger_inputs.proto_tensor" % run), float))
        else:
            all_input.append(tf.io.parse_tensor(tf.io.read_file("runs/%d/taggee_inputs.proto_tensor" % run), float))
    
        tagger_poses = tf.io.parse_tensor(tf.io.read_file("runs/%d/tagger_poses.proto_tensor" % run), float)
        taggee_poses = tf.io.parse_tensor(tf.io.read_file("runs/%d/taggee_poses.proto_tensor" % run), float)
    
        scores_normalizer = tf.math.sqrt(tf.cast(tf.math.square(IMG_INPUT_SHAPE[0]) + tf.math.square(IMG_INPUT_SHAPE[1]), float))
        
        if ROLE == "TAGGER":
            scores = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(tagger_poses - taggee_poses), axis=1))/scores_normalizer
        else:
            scores = (scores_normalizer - tf.math.sqrt(tf.math.reduce_sum(tf.math.square(tagger_poses - taggee_poses), axis=1)))/scores_normalizer
        
        if winner != ROLE:
            scores = tf.concat([scores[:-5:], tf.repeat(tf.constant([1.0]), repeats=5)], axis=0)
        all_scores.append(scores)
        print(run)

    all_next_frames = tf.concat([frame[1:] for frame in all_frames], axis=0)
    all_frames = tf.concat([frame[:-1] for frame in all_frames], axis=0)
    all_input = tf.cast(tf.concat([input[:-1] for input in all_input], axis=0), tf.float16)
    all_scores = tf.cast(tf.concat([scores[:-1] for scores in all_scores], axis=0), tf.float16)
    
    print(tf.shape(all_frames), tf.shape(all_next_frames), tf.shape(all_input), tf.shape(all_scores))
    
    big_train_ds = tf.data.Dataset.from_tensor_slices((all_frames, all_next_frames, all_input, tf.expand_dims(all_scores, axis=-1)))
    big_train_ds = big_train_ds.shuffle(big_train_ds.cardinality())
    big_valid_ds = big_train_ds.shard(8, 6)
    big_test_ds = big_train_ds.shard(8, 7)
    big_train_ds = big_train_ds.take((6 * big_train_ds.cardinality())//8)

1695099029
1695098660
1695098329
1695100432
1695100581
1695098876
1695099103
1695098364
1695098432
1695098474
1695100250
1695098472
1695099810
1695098637
1695098337
1695098372
1695098527
1695098388
1695100469
1695100199
1695098347
1695100607
1695098502
1695098542
1695100270
1695098724
1695100353
1695099903
1695098609
1695100371
tf.Tensor([12514   640   640     3], shape=(4,), dtype=int32) tf.Tensor([12514   640   640     3], shape=(4,), dtype=int32) tf.Tensor([12514     4], shape=(2,), dtype=int32) tf.Tensor([12514], shape=(1,), dtype=int32)


In [10]:
# Train cost calculator and encoder
from tensorflow.keras import optimizers

with tf.device("gpu:0"):
    enc_cost_calc = models.Sequential([layers.Input(shape=IMG_INPUT_SHAPE), encoder, cost_calculator])
    enc_cost_calc.compile(optimizers.Adam(learning_rate=ENCODER_AND_COST_LR), loss="mse", metrics=["mse"])
    enc_cost_calc.summary()
    
    frame_and_score = lambda w, x, y, z: (w, z)
    
    enc_cost_calc.fit(
        big_train_ds.map(frame_and_score, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False).batch(BATCH_SIZE),
        validation_data=big_valid_ds.map(frame_and_score, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False).batch(BATCH_SIZE),
        epochs=MAX_EPOCHS,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=MAX_EPOCHS//10, restore_best_weights=True)]
    )

encoder.trainable = False
cost_calculator.trainable = False

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32)                288804    
                                                                 
 sequential_2 (Sequential)   (None, 1)                 13057     
                                                                 
Total params: 301861 (1.15 MB)
Trainable params: 301797 (1.15 MB)
Non-trainable params: 64 (256.00 Byte)
_________________________________________________________________
Epoch 1/100


2023-09-28 21:01:09.358644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-09-28 21:01:09.507431: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-09-28 21:01:09.507464: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2023-09-28 21:01:09.507571: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-09-28 21:01:10.568753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-28 21:01:10.604657: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2238d5dcb

587/587 [==============================] - 67s 92ms/step - loss: 10.5939 - mse: 0.3506 - val_loss: 10.1062 - val_mse: 0.0728
Epoch 2/100
587/587 [==============================] - 25s 43ms/step - loss: 10.0172 - mse: 0.2000 - val_loss: 9.6334 - val_mse: 0.0369
Epoch 3/100
587/587 [==============================] - 25s 43ms/step - loss: 9.5128 - mse: 0.1433 - val_loss: 9.1656 - val_mse: 0.0273
Epoch 4/100
587/587 [==============================] - 25s 42ms/step - loss: 9.0127 - mse: 0.1110 - val_loss: 8.6828 - val_mse: 0.0221
Epoch 5/100
587/587 [==============================] - 24s 42ms/step - loss: 8.5077 - mse: 0.0925 - val_loss: 8.1873 - val_mse: 0.0211
Epoch 6/100
587/587 [==============================] - 25s 42ms/step - loss: 7.9910 - mse: 0.0766 - val_loss: 7.6784 - val_mse: 0.0188
Epoch 7/100
587/587 [==============================] - 24s 42ms/step - loss: 7.4672 - mse: 0.0645 - val_loss: 7.1618 - val_mse: 0.0189
Epoch 8/100
587/587 [==============================] - 24s 42ms/

In [ ]:
# Train prediction
with tf.device("cpu:0"):
    apply_encoder = lambda w, x, y, z: (tf.concat([tf.cast(encoder(w), tf.float16), y], axis=-1), tf.cast(encoder(x), tf.float16))
    pred_train_ds = big_train_ds.batch(BATCH_SIZE).map(apply_encoder, num_parallel_calls=BATCH_SIZE//4, deterministic=False).cache()
    pred_valid_ds = big_valid_ds.batch(BATCH_SIZE).map(apply_encoder, num_parallel_calls=BATCH_SIZE//4, deterministic=False).cache()

with tf.device("gpu:0"):
    predictor.compile(optimizers.Adam(learning_rate=PRED_LR), loss="mse", metrics=["mse"])
    predictor.summary()
    
    predictor.fit(
        pred_train_ds,
        validation_data=pred_valid_ds,
        epochs=MAX_EPOCHS,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=MAX_EPOCHS//10, restore_best_weights=True)]
    )

predictor.trainable=False

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                2368      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 32)               

In [12]:
# Train action generator
def minimize_loss(y_true, y_pred):
    return tf.math.reduce_mean(y_pred)


with tf.device("cpu:0"):
    apply_encoder = lambda w, x, y, z: (tf.cast(encoder(w), tf.float16), z)
    actgen_valid_ds = big_valid_ds.batch(BATCH_SIZE).map(apply_encoder, num_parallel_calls=BATCH_SIZE//4, deterministic=False).cache()
    actgen_train_ds = tf.data.Dataset.from_tensor_slices((tf.random.normal((100_000, OUTPUT_SIZE)), tf.zeros(100_000, 1))).batch(BATCH_SIZE)

model_in = layers.Input(shape=OUTPUT_SIZE)
action_gen_trainer = layers.Concatenate()([model_in, action_gen(model_in)])
action_gen_trainer = predictor(action_gen_trainer)
action_gen_trainer = cost_calculator(action_gen_trainer)
action_gen_trainer = models.Model(model_in, action_gen_trainer)

action_gen_trainer.compile(optimizers.Adam(learning_rate=ACT_GEN_LR), loss=minimize_loss, metrics=[minimize_loss])
action_gen_trainer.summary(show_trainable=True)

action_gen_trainer.fit(
    actgen_train_ds,
    validation_data=actgen_valid_ds,
    epochs=MAX_EPOCHS,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=MAX_EPOCHS//10, restore_best_weights=True)]
)

6250/6250 [==============================] - 127s 20ms/step - loss: 0.2706 - minimize_loss: 0.2605 - val_loss: 0.2722 - val_minimize_loss: 0.2621
Epoch 2/100
6250/6250 [==============================] - 74s 12ms/step - loss: 0.2704 - minimize_loss: 0.2604 - val_loss: 0.2722 - val_minimize_loss: 0.2620
Epoch 3/100
6250/6250 [==============================] - 74s 12ms/step - loss: 0.2705 - minimize_loss: 0.2604 - val_loss: 0.2722 - val_minimize_loss: 0.2620
Epoch 4/100
6250/6250 [==============================] - 74s 12ms/step - loss: 0.2705 - minimize_loss: 0.2604 - val_loss: 0.2722 - val_minimize_loss: 0.2620
Epoch 5/100
6250/6250 [==============================] - 74s 12ms/step - loss: 0.2704 - minimize_loss: 0.2603 - val_loss: 0.2722 - val_minimize_loss: 0.2620
Epoch 6/100
6250/6250 [==============================] - 74s 12ms/step - loss: 0.2705 - minimize_loss: 0.2604 - val_loss: 0.2722 - val_minimize_loss: 0.2620
Epoch 7/100
6250/6250 [==============================] - 74s 12ms/ste

KeyboardInterrupt: 

In [13]:
# Save model so it can play the game

pred_model = models.Sequential([layers.Input(shape=IMG_INPUT_SHAPE), encoder, action_gen])
pred_model.save("prediction_model_%s.keras" % ROLE)